In [20]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [21]:
ticker = input("Enter your ticker")
years = choice = int(input("Enter 5 or 10 for exit: "))

In [ ]:
##################################### Library and total-shares

url = f'https://www.screener.in/company/{ticker}/consolidated/'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extracting ratios
    ratios_section = soup.find('ul', id='top-ratios')
    market_cap = current_price = roe = None

    for item in ratios_section.find_all('li'):
        name = item.find('span', class_='name').get_text(strip=True)
        value = ''.join(item.find('span', class_='value').stripped_strings)

        # Extract only numbers using regex
        number_value = re.sub(r'[^\d.]', '', value)  # Allow decimals

        # print(value)
        if name == "Market Cap":
            market_cap = float(number_value)
        elif name == "Current Price":
            current_price = float(number_value)
        # elif name == "ROE":
        #     roe = float(number_value)/100  # Convert to float

    # Calculating total shares
    if market_cap and current_price:
        total_shares = market_cap / current_price
    else:
        total_shares = "N/A"

    # Print stored value
    # print("Current ROE (%):", roe*100)

else:
    print("Failed to retrieve data, Status Code:", response.status_code)

#########################################################################################################################################################################
###################################### PROFIT-GROWTH for growth rate #######################

# Check response status
if response.status_code == 200:

    # Initialize global variables
    profit_10yr = profit_5yr = None

    # Locate the table with "Return on Equity"
    tables = soup.find_all("table", class_="ranges-table")
    
    for table in tables:
        header = table.find("th").get_text(strip=True)
        profit = 69
        
        # Check if it's the "Return on Equity" table
        if "Compounded Profit Growth" in header:
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                if cells:
                    period = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True).replace("%", "")  # Remove % symbol
                    value = float(value)  # Convert to float

                    # Store values based on period
                    if period == "10 Years:":
                        profit_10yr = value/100
                    elif period == "5 Years:":
                        profit_5yr = value/100
        if years == 5 : 
            profit = profit_5yr
        elif years == 10 :
            profit = profit_10yr
else:
    print("Failed to retrieve data, Status Code:", response.status_code)

#########################################################################################################################################################################
################################# FREE CASH FLOW GROWTH RATE ################################
print(f'ticker : {ticker}')
def scrape_company_data(ticker, section_ids):
    url = f"https://www.screener.in/company/{ticker}/consolidated/#quarters"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Dictionary to store filtered dataframes for each section
    section_data = {}

    for section_id in section_ids:
        # Locate the section table
        section = soup.find("section", {"id": section_id})
        table = section.find("table", {"class": "data-table"}) if section else None
        
        if table:
            # Extract headers
            headers = [th.text.strip() for th in table.find("thead").find_all("th")]

            # Extract data
            data = []
            for row in table.find("tbody").find_all("tr"):
                cells = [cell.text.strip() for cell in row.find_all("td")]
                if len(cells) > 1:
                    data.append(cells)

            # Create DataFrame if data is available
            if data:
                df = pd.DataFrame(data, columns=headers)
                section_data[section_id] = df
        else:
            print(f"⚠️ No table found for section ID: {section_id}")

    return section_data

# Example Usage
section_ids = ["profit-loss", "cash-flow", "balance-sheet"]
dataframes = scrape_company_data(ticker, section_ids)

# ✅ Extract CFO (Last 5 Years)
df_cash_flow = dataframes.get("cash-flow")
cfo_row = df_cash_flow[df_cash_flow.iloc[:, 0].str.contains("Cash from Operating Activity", case=False, na=False)]

if not cfo_row.empty:
    cfo_values = cfo_row.iloc[0, 1:].apply(lambda x: float(x.replace(",", "")) if x else None).tolist()
    cfo_last_5_years = cfo_values[-5:]  # last 5 values
else:
    print("⚠️ Could not find 'Cash from Operating Activity' row.")

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json, text/javascript, */*; q=0.01",
}

def get_internal_ticker_id(stock_symbol, headers):
    """Fetch the internal ticker ID from Screener"""
    company_url = f"https://www.screener.in/company/{stock_symbol}/consolidated/"
    response = requests.get(company_url, headers=headers)

    if response.ok:
        soup = BeautifulSoup(response.content, "html.parser")
        company_info_div = soup.find("div", {"id": "company-info"})
        
        # Extract company ID
        if company_info_div:
            ticker_id = company_info_div.get("data-company-id")
            if ticker_id:
                # print(ticker_id)
                return ticker_id

def fetch_cash_flow_data(stock_symbol, headers):
    """Fetch last 5 values of 'Fixed assets purchased' from Screener"""
    ticker_id = get_internal_ticker_id(stock_symbol, headers)
    
    if ticker_id:
        api_url = f"https://www.screener.in/api/company/{ticker_id}/schedules/?parent=Cash+from+Investing+Activity&section=cash-flow&consolidated="
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            fixed_assets_data = data.get('Fixed assets purchased', {})

            if fixed_assets_data:
                sorted_years = sorted(fixed_assets_data.keys())[-5:]
                last_5_values = []

                for year in sorted_years:
                    raw_value = fixed_assets_data[year].replace(",", "")
                    try:
                        value = float(raw_value)
                        last_5_values.append(value)
                    except ValueError:
                        last_5_values.append(None)

                
                return last_5_values

    print("Failed to fetch CAPEX data.")
    return []

# Example Usage
capex_array = fetch_cash_flow_data(ticker, HEADERS)
# print("CAPEX array (last 5 years):", capex_array)

freecash = []
for i in range (0,5):
    val = cfo_last_5_years[i] - capex_array[i]
    freecash.append(val)
# print(f'Last 5 years FCF : {freecash}')

def adjusted_cagr(values):
    # Find the first positive value as new starting point
    for i in range(len(values) - 1):
        if values[i] > 0:
            initial = values[i]
            final = values[-1]
            years = len(values) - 1 - i
            break
    else:
        return None  # All values are non-positive; CAGR not possible

    cagr = (final / initial) ** (1 / years) - 1
    return cagr * 100

cagr_result = adjusted_cagr(freecash)/100
# print(f"Free cash flow CAGR: {cagr_result:.2f}%")

#########################################################################################################################################################################



ticker : INFY


In [51]:
FCF = freecash[-1]
discount = 6.1/100
terminal = 3/100

In [52]:
# Display the extracted ticker data
print(f'Ticker : {ticker}')
print("Market Cap:", market_cap)
print("Current Price:", current_price)
print("Total number of shares:", total_shares)
print("\n")

# Display the extracted ROE values
print("Profit-growth 5 Years(%):", profit_5yr*100)
print("Profit-growth 10 Years(%):", profit_10yr*100)
print("Profit-growth for calculation(%):", profit*100)
print("\n")

# Display the extracted FCF valuec
print(f"CFO (Last 5 Years): {cfo_last_5_years}")
print("CAPEX array (last 5 years):", capex_array)
print(f'Last 5 years FCF : {freecash}')
print(f"Free cash flow CAGR(%): {cagr_result*100:.2f}")
print("\n")

# Display the assumed rates
print("Profit-growth(%):", profit*100)
print(f"Free cash flow CAGR(%): {cagr_result*100:.2f}")
print("Discount rate(%):", discount*100)
print("Terminal Growth rate(%):", terminal*100)


Ticker : INFY
Market Cap: 582994.0
Current Price: 1404.0
Total number of shares: 415.23789173789174


Profit-growth 5 Years(%): 11.0
Profit-growth 10 Years(%): 9.0
Profit-growth for calculation(%): 11.0


CFO (Last 5 Years): [17003.0, 23224.0, 23885.0, 22467.0, 25210.0]
CAPEX array (last 5 years): [-3307.0, -2107.0, -2161.0, -2579.0, -2201.0]
Last 5 years FCF : [20310.0, 25331.0, 26046.0, 25046.0, 27411.0]
Free cash flow CAGR(%): 7.78


Profit-growth(%): 11.0
Free cash flow CAGR(%): 7.78
Discount rate(%): 6.1
Terminal Growth rate(%): 3.0


In [53]:
############### DCF CALCULATION

# Compute terminal value
def calculate_terminal_value(fcf_initial, growth_rate, long_growth_rate, wacc, years):
    # Calculate FCFs for the last 3 years
    fcf_years = [fcf_initial * ((1 + growth_rate) ** (years - i)) for i in range(3)]
    avg_fcf_last_3_years = sum(fcf_years) / 3

    # Calculate terminal value using the average
    terminal_value = (avg_fcf_last_3_years * (1 + long_growth_rate)) / (wacc - long_growth_rate)
    return terminal_value


terminal_value1 = calculate_terminal_value(FCF, profit, terminal, discount, years)
terminal_value2 = calculate_terminal_value(FCF, cagr_result, terminal, discount, years)
# print(terminal_value)

def discounted_cash_flow(fcf_initial, growth_rate, wacc, years, terminal_value):
    """
    Calculate the discounted cash flow (DCF) valuation.
    
    Returns:
    - Present value of the firm
    """
    fcf_list = [fcf_initial * ((1 + growth_rate) ** t) for t in range(1, years + 1)]
    # print(f'Projected FCF : {fcf_list}')
    
    # DCF value for forecasted years
    dcf_value = sum(fcf / ((1 + wacc) ** (t + 1)) for t, fcf in enumerate(fcf_list))
    
    # Discount terminal value
    discounted_terminal_value = terminal_value / ((1 + wacc) ** years)
    
    return dcf_value + discounted_terminal_value

# Example usage:
dcf_valuation1 = discounted_cash_flow(FCF, profit, discount, years, terminal_value1)
dcf_valuation2 = discounted_cash_flow(FCF, cagr_result, discount, years, terminal_value2)
equity1 = dcf_valuation1                   
equity2 = dcf_valuation2 


# Calculate fair prices using two different growth rates
fair_price1 = equity1 / total_shares if total_shares else 0
fair_price2 = equity2 / total_shares if total_shares else 0

# Ensure lower price is shown first
low_fair, high_fair = sorted([fair_price1, fair_price2])

# Calculate safe prices with margin of safety.
safe = 50/100
safe_price1 = low_fair * (1 - safe)
safe_price2 = high_fair * (1 - safe)

# Output
print(f"\n🎯 Fair Price Range: ₹{low_fair:.2f} - ₹{high_fair:.2f}")
print(f"🛡️ Safe Price Range (with {int(safe * 100)}% margin of safety): ₹{safe_price1:.2f} - ₹{safe_price2:.2f}")


🎯 Fair Price Range: ₹2551.86 - ₹2864.09
🛡️ Safe Price Range (with 50% margin of safety): ₹1275.93 - ₹1432.05
